In [ ]:
from glob import glob
import os

h5files = glob("/home/hulabdl/CMRxRecon2025/h5_dataset/*.h5")

groups = dict()

for fname in h5files:
    basename = fname.split("/")[-1]
    groupname = "@".join(basename.split("@")[:-2])
    groups.setdefault(groupname, []).append(fname)


nsample_for_validation_each_group = 1

groups_val = dict()

for groupname, files in groups.items():
    for i in range(nsample_for_validation_each_group):
        if i < len(files):
            groups_val.setdefault(groupname, []).append(files.pop(0))

for groupname, files in groups_val.items():
    print(f"Group: {groupname}, Validation Files: {len(files)}")
    for fname in files:
        print(f"  - {fname}")

train_dir = "/home/hulabdl/CMRxRecon2025/maxsplitset/train/"
train_fnamelist = [fname for files in groups.values() for fname in files]


val_dir = "/home/hulabdl/CMRxRecon2025/maxsplitset/val/"
val_fnamelist = [fname for files in groups_val.values() for fname in files]


def create_symlinks(fnamelist, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    for fname in fnamelist:
        basename = os.path.basename(fname)
        symlink_path = os.path.join(target_dir, basename)
        if not os.path.exists(symlink_path):
            os.symlink(fname, symlink_path)
            print(f"Created symlink: {symlink_path}")
        else:
            print(f"Symlink already exists: {symlink_path}")

create_symlinks(train_fnamelist, train_dir)
create_symlinks(val_fnamelist, val_dir)

print("Symlinks created successfully.")

In [ ]:
# DumpFilelist
from glob import glob
import os

output_to = "/home/hulabdl/CMRxRecon2025//maxsplitset"

fnamelist = glob("/home/hulabdl/CMRxRecon2025//maxsplitset/train/*.h5")

basenamelist = [os.path.basename(fname) for fname in fnamelist]

with open(os.path.join(output_to, "train_filelist.txt"), "w") as f:
    for basename in basenamelist:
        f.write(f"{basename}\n")

fnamelist = glob("/home/hulabdl/CMRxRecon2025//maxsplitset/val/*.h5")

basenamelist = [os.path.basename(fname) for fname in fnamelist]

with open(os.path.join(output_to, "val_filelist.txt"), "w") as f:
    for basename in basenamelist:
        f.write(f"{basename}\n")

print("File lists created successfully.")

In [ ]:
# Load filelist

from glob import glob
import os

src_dir = "/home/hulabdl/CMRxRecon2025/h5_dataset"
src_fnamelist = "/home/hulabdl/CMRxRecon2025/maxsplitset/val_filelist.txt"

tgt_dir = "/home/hulabdl/CMRxRecon2025/maxsplitset/val/"

fullpath_list = glob(os.path.join(src_dir, "*.h5"))

with open(src_fnamelist, 'r') as f:
    for fname in f:
        fname = fname.strip()
        once = True
        for file in fullpath_list:
            if fname in file:
                assert once, "Only one file should match"
                if not os.path.exists(os.path.join(tgt_dir, fname)):
                    os.symlink(file, os.path.join(tgt_dir, fname))
                once = False

In [ ]:
import os
import random

def create_symlinks(source_dir, target_dir, num_files=5):
    # 检查源路径是否存在
    if not os.path.exists(source_dir):
        raise FileNotFoundError(f"源路径 {source_dir} 不存在")
    
    # 如果目标路径不存在，则创建
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    
    # 获取源路径中的所有文件
    all_files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]
    
    # 如果文件数量不足，抛出错误
    if len(all_files) < num_files:
        raise ValueError(f"源路径中的文件不足 {num_files} 个，仅有 {len(all_files)} 个。")
    
    # 随机选择 num_files 个文件
    selected_files = random.sample(all_files, num_files)
    
    # 在目标路径中为选中的文件创建软链接
    for file_name in selected_files:
        source_file = os.path.join(source_dir, file_name)
        target_file = os.path.join(target_dir, file_name)
        
        try:
            os.symlink(source_file, target_file)
            print(f"已创建软链接: {target_file} -> {source_file}")
        except FileExistsError:
            print(f"目标文件已存在，跳过: {target_file}")
        except OSError as e:
            print(f"创建软链接失败: {e}")

# 示例使用
source_path = "/home_data/home/liyuyang/data2/datasets/CMRxRecon2025/h5_dataset@"  # 源路径
target_path = "/home_data/home/liyuyang/data2/datasets/CMRxRecon2025/minimunset/train"  # 目标路径

create_symlinks(source_path, target_path, num_files=7)